In [18]:
%matplotlib ipympl
import sqlite3 as sql
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipyleaflet import Map, Polyline, Marker, Popup, basemaps, WidgetControl
from IPython.display import display, clear_output
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.colors
import pandas as pd

output_widget_compare = widgets.Output()
output_widget_fetch_data = widgets.Output()
output_widget_plot_all = widgets.Output()
output_widget_plot_selected = widgets.Output()
output_widget_search = widgets.Output()

# Connect to the SQLite database
conn = sql.connect('flights.db')
cur = conn.cursor()

# Select the required columns from the airport_data table
query = "SELECT Airport, lat_decimal, lon_decimal FROM airport_data;"
cur.execute(query)
all_destinations_data = cur.fetchall()

# Close the connection to the database
conn.close()

# Convert the results into a DataFrame
flights_df = pd.DataFrame(all_destinations_data, columns=['Airport', 'lat_decimal', 'lon_decimal'])


# Get a list of unique airports
unique_airports = flights_df['Airport'].unique()

# Create a colormap
n_colors = len(unique_airports)
colormap = cm.get_cmap('tab20', n_colors)

# Generate colors
colors = [colormap(i) for i in range(n_colors)]
color_mapping = dict(zip(unique_airports, colors))

# Function to update the output widget with the plot
def update_output(fig, output_widget):
    with output_widget:
        clear_output(wait=True)
        display(fig)
        
# Define a dictionary to store the colors of routes on the map
route_colors = {}

# Function to create a blue polyline on the map
def create_blue_polyline(airport, coordinates):
    if any(coord is None for coord in coordinates):
        print(f"Invalid coordinates for {airport}: {coordinates}")
        return None

    color_hex = "blue"
    print(f"Creating blue polyline for {airport}")

    polyline = Polyline(locations=coordinates, color=color_hex, fill=False)
    route_colors[airport] = color_hex  # Store the color in the dictionary

    def on_click(event=None, properties=None, **args):
        global selected_airports
        if airport not in selected_airports:
            airport_select.value = list(selected_airports) + [airport]

    polyline.on_click(on_click)
    return polyline
'''
def compareAirports(airports, year=None):   # the input for this function is a list called "airports"
    con = sql.connect("flights.db")
    fig, ax = plt.subplots(figsize=(12, 6))

    # this section queries the database to generate a df used to generate the time series for each 
    for airport in airports:
        airport_param = " " + airport.strip() + " "
        
        if year is not None:
            query = "SELECT Time, No FROM flight_data WHERE Type='PAS' AND Airport=? AND YEAR=? AND AMOUNT = 'Total' ORDER BY Time"
            params = [airport_param, year]
            
            
        else:    
            query = "SELECT Time, No FROM flight_data WHERE Type='PAS' AND Airport=? AND AMOUNT = 'Total' ORDER BY Time"
            df = pd.read_sql_query(query, con, params=[airport_param])
            df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m', errors='coerce')
            params = [airport_param]

        if df.empty:
            print(f"No data available for airport: {airport}")
            continue

        ax.plot(df['Time'], df['No'], label=airport)

    con.close()
    
    ax.set_title('Passenger Data', fontsize=18)
    ax.set_ylabel('# Passengers', fontsize=16)
    ax.set_ylim(bottom=0)
    ax.tick_params(axis='both', labelsize=14)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    plt.grid(True, axis='y')

    # Put a legend to the right of the current axis
    ax.legend(title = "Destination Airport(s)", loc='center left', bbox_to_anchor=(1, 0.5))
    
    update_output(fig, output_widget_compare)
'''

def update_map():
    # Clear existing polylines from the map
    for layer in list(m.layers):
        if isinstance(layer, Polyline):
            m.remove_layer(layer)

    # Add new polylines for currently selected airports
    for airport in selected_airports:
        airport_data = flights_df[flights_df['Airport'] == airport]
        for _, row in airport_data.iterrows():
            path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
            polyline = create_polyline(airport, path, source='dropdown')
            m.add_layer(polyline)
def compareAirports(airports, year=None):
    con = sql.connect("flights.db")
    fig, ax = plt.subplots(figsize=(32, 27))

    for airport in airports:
        plot_color = color_mapping.get(airport, "black")
        airport_param = f"%{airport.strip()}%"  # Use the LIKE operator for a broader match
        query = """
        SELECT Time, No FROM flight_data
        WHERE Type='PAS' AND Airport LIKE ? AND AMOUNT = 'Total'
        
        """
        params = [airport_param]

        if year is not None:
            print(f"Querying for airport: {airport} in year: {year}")
            query += " AND strftime('%Y', Time) = ?"
            params.append(str(year))
            datefmt = mdates.DateFormatter('%b')
            ax.xaxis.set_major_formatter(datefmt)
            ax.set_title(f"Passenger Data for {year}", fontsize=60)
            
        else:
            ax.set_title('Passenger Data for All Years', fontsize=60)
        
        query += " ORDER BY Time"
        df = pd.read_sql_query(query, con, params=params)

        # Convert 'Time' to datetime, assuming the format is 'YYYY-MM-DD HH:MM:SS'
        df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

        # Remove rows with NaT in 'Time' column
        df = df.dropna(subset=['Time'])

        if df.empty:
            print(f"No data available for airport: {airport}")
            continue

        ax.plot(df['Time'], df['No'], label=airport, color=plot_color)

    con.close()
    ax.set_ylabel('# Passengers', fontsize=45)
    ax.set_ylim(bottom=0)
    ax.tick_params(axis='both', labelsize=40)
    ax.legend(title="Destination Airport(s)", loc='center left', bbox_to_anchor=(1, 0.5), fontsize=45)
    update_output(fig, output_widget_compare)
  

    # Put a legend to the right of the current axis
    ax.legend(title="Destination Airport(s)", loc='center left', bbox_to_anchor=(1, 0.5), fontsize=45)
    
    update_output(fig, output_widget_compare)


def plot_all(airport):
    con = sql.connect("flights.db")
    # Add spaces before and after the airport name
   
    query = "SELECT Airport, Time, No FROM flight_data WHERE Type='PAS' AND AMOUNT = 'Total' ORDER BY Time"
    df1 = pd.read_sql_query(query, con)
    con.close()

    if df1.empty:
        print(f"No data available for airport: {airport}")
        return
    

    df1['Time'] = pd.to_datetime(df1['Time'], format='%Y-%m', errors='coerce')
    if df1['Time'].isnull().any():
        print("Some dates were not parsed correctly")
        
    fig, ax = plt.subplots(figsize=(32, 27))

    for airport in df1['Airport'].unique():
        airport_df = df1[df1['Airport'] == airport]
        ax.plot(airport_df['Time'], airport_df['No'], label=airport.strip())

    plt.title('Passenger Data for All Airports', fontsize=45)
    ax.set_ylabel('# Passengers', fontsize=35)
    ax.set_ylim(bottom=0)
    ax.tick_params(axis='both', labelsize=40)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(title = "Destination Airport(s)", loc='center left', bbox_to_anchor=(1, 0.5))
    update_output(fig, output_widget_plot_all)

year_dropdown = widgets.Dropdown(
    options=[('All Years', None)] + [(str(year), year) for year in range(2016, 2024)],
    value=None,
    description='Year:',
    disabled=False,
)

# Connect to the SQLite database
conn = sql.connect('flights.db')
cur = conn.cursor()

# Select the required columns from the airport_data table
query = "SELECT Airport, lat_decimal, lon_decimal FROM airport_data;"
cur.execute(query)
all_destinations_data = cur.fetchall()

# Close the connection to the database
conn.close()


# Convert the results into a DataFrame
flights_df = pd.DataFrame(all_destinations_data, columns=['Airport', 'lat_decimal', 'lon_decimal'])

# Coordinates for Dublin
dublin_coords = (53.4268, -6.2562)

m = Map(center=dublin_coords, zoom=6, basemap=basemaps.OpenStreetMap.Mapnik) 

selected_airports = []
selected_polylines = {}

# Modified 'create_polyline' to include a source parameter
def create_polyline(airport, coordinates, source='slider'):
    if any(coord is None for coord in coordinates):
        print(f"Invalid coordinates for {airport}: {coordinates}")
        return None

    color_hex = "blue" if source == 'slider' else matplotlib.colors.to_hex(color_mapping.get(airport, "black"))

    polyline = Polyline(locations=coordinates, color=color_hex, fill=False)
    hover_popup = Popup(location=coordinates[-1], child=widgets.HTML(value=f"<b>{airport}</b>"))
    polyline.popup = hover_popup

    def on_click(event=None, properties=None, **args):
        global selected_airports, selected_polylines
        if polyline.color == "blue":
            polyline.color = matplotlib.colors.to_hex(color_mapping.get(airport, "black"))
            selected_airports.append(airport)
        else:
            polyline.color = "blue"
            if airport in selected_airports:
                selected_airports.remove(airport)
        polyline.style = {'color': polyline.color}

    polyline.on_click(on_click)
    return polyline



clear_selection_button = widgets.Button(description="Clear Selection")

# Function to clear the selected routes
def clear_selection(b):
    global selected_polylines
    for polyline in selected_polylines.values():
        polyline.color = "blue"
        polyline.style = {'color': "blue"}
    selected_polylines.clear()
    # Also clear the selection in the airport_select widget
    airport_select.value = []

clear_selection_button.on_click(clear_selection)
     


def plot_selected_airports(b):
    selected_year = year_dropdown.value
    if len(selected_airports) > 0:  # Check if the list is not empty
        compareAirports(selected_airports, year=selected_year)
    else:
        with output_widget_compare:
            clear_output(wait=True)

plot_button = widgets.Button(description="Plot Selected Airports")
plot_button.on_click(plot_selected_airports)

                  

# Create a slider widget for selecting the number of airports
num_airports_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=len(flights_df),
    step=1,
    description='Number of Airports:',
    continuous_update=False
)

# Create a dropdown menu for selecting specific airports with an increased size
airport_select = widgets.SelectMultiple(
    options=flights_df['Airport'].unique().tolist(),
    value=[flights_df['Airport'].unique()[0]],
    description='Airports:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')  # Adjusted size
)


def on_slider_change(change):
    # Display polylines for the number of airports specified by the slider
    # without affecting the selected_airports list.
    displayed_airports = flights_df['Airport'].unique()[:change['new']]

    # Clear the existing polylines from the map
    for layer in list(m.layers):
        if isinstance(layer, Polyline):
            m.remove_layer(layer)

    # Add new polylines for the displayed airports
    for airport in displayed_airports:
        airport_data = flights_df[flights_df['Airport'] == airport]
        for _, row in airport_data.iterrows():
            path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
            polyline = create_polyline(airport, path)
            m.add_layer(polyline)

    # Update the map with new polylines
    update_map()






def update_map_with_slider_airports(airports):
    # Clear the existing slider-controlled polylines from the map
    for layer in list(m.layers):
        if isinstance(layer, Polyline) and layer not in selected_polylines.values():
            m.remove_layer(layer)

    # Add new polylines for the slider-controlled airports
    for airport in airports:
        if airport not in selected_airports:  # Avoid duplicating polylines for selected airports
            airport_data = flights_df[flights_df['Airport'] == airport]
            for _, row in airport_data.iterrows():
                path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
                polyline = create_polyline(airport, path)
                m.add_layer(polyline)

def on_slider_change(change):
    displayed_airports = flights_df['Airport'].unique()[:change['new']]
    update_map_with_slider_airports(displayed_airports)

num_airports_slider.observe(on_slider_change, names='value')

def on_airport_dropdown_change(change):
    global selected_airports, selected_polylines
    # Update the selected airports list
    selected_airports = list(change['new'])

    # Update the map with the selected airports
    update_map_with_dropdown_airports(selected_airports)

def on_airport_change(change):
    global selected_airports, selected_polylines
    selected_airports = list(change['new'])

    # Update the map with polylines for the newly selected airports
    update_map()

airport_select.observe(on_airport_change, names='value')
    
def display_graph_side_by_map():
    graph_widget = widgets.VBox([output_widget_compare, output_widget_plot_all, full_screen_button])
    widget_control = WidgetControl(widget=graph_widget, position='bottomright', max_width=300, max_height=400)
    m.add_control(widget_control)

    clear_selection_button = widgets.Button(description="Clear Selection")

def clear_selection(b):
    global selected_polylines, selected_airports
    for polyline in selected_polylines.values():
        polyline.color = "blue"
        polyline.style = {'color': "blue"}
    selected_polylines.clear()
    selected_airports.clear()
    airport_select.value = []

clear_selection_button.on_click(clear_selection)

update_map()  # Corrected this line

plot_all_button = widgets.Button(description="Plot all Airports")
plot_all_button.on_click(plot_all)
plot_button = widgets.Button(description="Plot Selected Airports")
plot_button.on_click(plot_selected_airports)

num_airports_slider.observe(on_slider_change, names='value')
airport_select.observe(on_airport_change, names='value')
# Create a search bar to filter airports
search_input = widgets.Text(
    value='',
    placeholder='Type to search airports',
    description='Search:',
    disabled=False
)

# Store all available airports in a list
all_airports = flights_df['Airport'].unique().tolist()

# Initialize a global list to keep track of selected airports across searches
global_selected_airports = []

output_widget_search = widgets.Output()

# Function to update the airport options based on search input
def update_airport_options(change):
    global global_selected_airports
    search_query = change['new']

    with output_widget_search:
        clear_output(wait=True)  # Clear existing content
        try:
            if search_query:  # Check if search_query is not None and not empty
                # Filter airports matching the search query
                filtered_airports = [airport for airport in all_airports if search_query.lower() in airport.lower()]
            else:
                filtered_airports = all_airports  # If search_query is None or empty, use all airports

            # Ensure currently selected airports are also included
            combined_options = sorted(set(filtered_airports + list(global_selected_airports)), key=lambda x: all_airports.index(x))

            # Update the options and reapply valid selections
            airport_select.options = combined_options
            valid_selections = [airport for airport in global_selected_airports if airport in combined_options]
            airport_select.value = valid_selections

        except Exception as e:
            print(f"Error during search: {e}")


# Make sure to observe the search input changes
search_input.observe(update_airport_options, names='value')



# Function to update the global selected airports when the user changes the selection
def on_airport_select_change(change):
    global selected_airports, selected_polylines
    selected_airports = list(change['new'])
# Observe changes in the search input and update the airport options accordingly
search_input.observe(update_airport_options, names='value')

# Observe changes in the airport_select widget to update the global selected airports
airport_select.observe(on_airport_select_change, names='value')

def update_map_with_slider_airports(airports):
    for layer in list(m.layers):
        if isinstance(layer, Polyline) and layer not in selected_polylines.values():
            m.remove_layer(layer)

    for airport in airports:
        if airport not in selected_airports:
            airport_data = flights_df[flights_df['Airport'] == airport]
            for _, row in airport_data.iterrows():
                path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
                polyline = create_polyline(airport, path, source='slider')
                m.add_layer(polyline)
                
            

# New function to update map based on dropdown
def update_map_with_dropdown_airports(airports):
    for airport in airports:
        airport_data = flights_df[flights_df['Airport'] == airport]
        for _, row in airport_data.iterrows():
            path = [(dublin_coords[0], dublin_coords[1]), (row['lat_decimal'], row['lon_decimal'])]
            polyline = create_polyline(airport, path, source='dropdown')
            m.add_layer(polyline)
            
num_airports_slider.observe(on_slider_change, names='value')
airport_select.observe(on_airport_dropdown_change, names='value')      

# Modify the layout to include the search output widget
layout = widgets.VBox([
    widgets.Label('Select Airports and Year for Analysis'),
    output_widget_search,  # Output widget for search results
    widgets.HBox([num_airports_slider, airport_select, clear_selection_button], layout=widgets.Layout(padding='10px')),
    year_dropdown,
    widgets.HBox([plot_button, plot_all_button], layout=widgets.Layout(justify_content='space-around'))
], layout=widgets.Layout(align_items='center'))
# Prepare the graph widget
graph_widget = widgets.VBox([output_widget_compare, output_widget_plot_all], layout=widgets.Layout(width='40%'))  # 40% width for the graph

# Prepare the map widget with a slightly larger width
map_widget = widgets.VBox([m], layout=widgets.Layout(width='60%'))  # 60% width for the map

# Create an HBox for side-by-side layout
side_by_side_layout = widgets.HBox([map_widget, graph_widget])

# Display the combined layout
display(layout)  # This displays the control widgets
display(side_by_side_layout)  # This displays the map and graph side by side

C:\Users\domin\AppData\Local\Temp\ipykernel_26688\2869930640.py:39: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('tab20', n_colors)


TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.